### Lister tous les fichiers audio avec leurs labels

In [22]:
import os 
import glob

genres = sorted(os.listdir("../Data/genres_original"))  


filepaths = []
labels = []

for idx , genre in enumerate(genres):
    files = glob.glob(f"../Data/genres_original/{genre}/*.wav")
    for f in files :
        filepaths.append(f)
        labels.append(idx)

### Découper en train / val / test

In [23]:
from sklearn.model_selection import train_test_split


X_temp, X_test, y_temp, y_test = train_test_split(
    filepaths, labels, test_size=0.15, random_state=42, stratify=labels)


X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1765, random_state=42, stratify=y_temp)
 

print(len(X_train), "train")
print(len(X_val),   "val")
print(len(X_test),  "test")


699 train
151 val
150 test


### découper les fichiers en segments

In [24]:
import librosa
import audioread  # force fallback

def split_audio(file, segment_duration=3, sr=22050):
    try:
        signal, _ = librosa.load(file, sr=sr)
    except Exception as e:
        print("Erreur sur:", file, e)
        return []  # ignorer le fichier problématique

    samples_per_segment = sr * segment_duration
    segments = []

    for start in range(0, len(signal), samples_per_segment):
        end = start + samples_per_segment
        part = signal[start:end]
        if len(part) == samples_per_segment:
            segments.append(part)

    return segments


### Appliquer la découpe à train

In [25]:
X_train_segments = []
y_train_segments = []

for idx, file in enumerate(X_train):
    segs = split_audio(file)
    X_train_segments.extend(segs)
    y_train_segments.extend([y_train[idx]] * len(segs))

print("Train segments:", len(X_train_segments))


/var/folders/rg/r64x99yx2k91vgyysnwnb7pc0000gn/T/ipykernel_4939/2648669402.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, _ = librosa.load(file, sr=sr)
/Users/mac/Desktop/Ast_Projet/.venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Erreur sur: ../Data/genres_original/jazz/jazz.00054.wav 
Train segments: 6977


### Appliquer la découpe à test

In [26]:
X_test_segments = []
y_test_segments = []

for idx, file in enumerate(X_test):
    segs = split_audio(file)
    X_test_segments.extend(segs)
    y_test_segments.extend([y_test[idx]] * len(segs))

print("Test segments:", len(X_test_segments))


Test segments: 1495


### Appliquer la découpe à val

In [27]:
X_val_segments = []
y_val_segments = []

for idx, file in enumerate(X_val):
    segs = split_audio(file)
    X_val_segments.extend(segs)
    y_val_segments.extend([y_val[idx]] * len(segs))

print("Val segments:", len(X_val_segments))


Val segments: 1509


### convertir un segment → log-mel spectrogram